# Markov Chain Simulation of a Mobile Cellular Network
Consider the 49-cell cellular network model with the wrapped-around design depicted in the below figure. Choose your own input parameters for the number of channels per cell, arrival rates, and mean holding times. Use a Markov-chain simulation to approximate the overall blocking and dropping probabilities in this network. For the same 49-cell network, obtain the blocking probability using EFPA. Compare the results of the two approaches for a range of parameter values.

![jupyter](./images/cellular_network.png)

In [22]:
import random as random
import numpy as np
import scipy.stats as stats

## Model Generation
A better illustration of the pseudo-code of the slides:

```
while sum(Na) < MAXNa:
    if R1(01) < sum(lambda) / (sum(lambda) + sum(Q * mu) + sum(Q * delta)):
        for i in m:
            if R2(01) <= sum(lambda[0:i]) / sum(lambda)
                Na[i]++
                if Q[i] < k:
                    Q[i]++
                else:
                    Nb[i]++
                break
    else if R1(01) < (sum(lambda) + sum(Q * mu)) / (sum(lambda) + sum(Q * mu) + sum(Q * delta)):
        for i in m:
            if R2(01) <= sum(Q[0:i] * mu) / sum(Q * mu):
                Q[i]--
                break
    else:
        for i in m:
            if R2(01) <= sum(Q[0:i] * delta[0:i]) / sum(Q * delta):
                Q[i]--
                from_cell = i
                break
        for i in Neig[from_cell].length:
            if R3(01) <= sum(P(from_cell, 0:i)):
                to_cell = Neig[from_cell][i]
                Nh[to_cell]++
                if Q[to_cell] == k:
                    Nd[to_cell]++
                else:
                    Q[to_cell]++
                break
Pb = sum(Nb) / MAXNa
Pd = sum(Nd) / sum(Nh)
```
Be aware that every R(01) in their loops should remain the same.

In [23]:
def markov_chain_cellular_network(num_cell, total_arrivals, mu, lam, handover_probability, delta, k, neighbours):
    queue_size = np.zeros(num_cell) # Q(i)
    blocking_probability = 0.0 # Pb
    dropping_probability = 0.0 # Pd
    num_current_arrivals = np.zeros(num_cell) # Na
    num_blocked_new_calls = np.zeros(num_cell) # Nb
    num_handovers = np.zeros(num_cell) # Nh
    num_drop = np.zeros(num_cell) # Nd
    neighbours_size = 6

    while np.sum(num_current_arrivals) < total_arrivals:
        ran = random.random()
        # arrival
        if ran <= np.sum(lam) / (np.sum(lam) + np.sum(queue_size * mu) + np.sum(queue_size * delta)):
            # find out in which of the cell the arrival occurs
            ran_2 = random.random()
            for i in range(num_cell):
                if ran_2 <= np.sum(lam[0 : i + 1]) / np.sum(lam):
                    num_current_arrivals[i] += 1
                    if queue_size[i] < k:
                        queue_size[i] += 1
                    else:
                        num_blocked_new_calls[i] += 1
                    break
        # departure
        elif ran <= (np.sum(lam) + np.sum(queue_size * mu)) / (np.sum(lam) + np.sum(queue_size * mu) + np.sum(queue_size * delta)):
            # find out in which of the cell the departure occurs
            ran_2 = random.random()
            for i in range(num_cell):
                if ran_2 <= np.sum(queue_size[0 : i + 1] * mu) / np.sum(queue_size * mu):
                    queue_size[i] -= 1
                    break
        # handover
        else:
            # find out from which of the cell it handovers out of (from_cell)
            ran_2 = random.random()
            from_cell = -1
            for i in range(num_cell):
                if ran_2 <= np.sum(queue_size[0 : i + 1] * delta[0 : i + 1]) / np.sum(queue_size * delta):
                    queue_size[i] -= 1
                    from_cell = i # cell index
                    break
            # find out into which cell the call handover in (to_cell)
            ran_3 = random.random()
            neighbour = neighbours[from_cell + 1]
            prob_i_n = handover_probability[from_cell]
            for j in range(neighbours_size):
                if ran_3 <= np.sum(prob_i_n[0 : j + 1]):
                    to_cell = neighbour[j] - 1 # cell index
                    num_handovers[to_cell] += 1
                    if queue_size[to_cell] == k:
                        num_drop[to_cell] += 1
                    else:
                        queue_size[to_cell] += 1
                    break

    blocking_probability = np.sum(num_blocked_new_calls) / total_arrivals
    dropping_probability = np.sum(num_drop) / np.sum(num_handovers)
    return blocking_probability, dropping_probability

For the parameters, to compare the results with EFPA, I set all the parameters of the cells to the same. They are: $\mu=2$, $\lambda=6$, $\delta=0.2$, $k=10$, $P\left(i,n\right)=\frac{1}{6}$, $MAXN_a=100,000$.

To obtain the confidence interval of the blocking probability, I ran the simulation program 25 times. Below is the result.

The blocking probability is 0.0008052, with a confidence interval of (0.0007695753518549943, 0.0008408246481450055).

In [25]:
num_cell = 49 # m
total_arrivals = 100000 # MAXNa
mu = 2
lam = np.ones(num_cell) * 6
handover_probability = np.ones((num_cell, 6)) / 6
delta = np.ones(num_cell) * 0.2
k = 10
neighbours = {1:[2,7,8,9,46,47],
              2:[1,3,9,10,47,48],
              3:[2,4,10,11,48,49],
              4:[3,4,5,11,12,43,49],
              5:[4,6,12,13,43,44],
              6:[5,7,13,14,44,45],
              7:[1,6,8,14,45,46],
              8:[1,7,9,14,15,21],
              9:[1,2,8,10,15,16],
              10:[2,3,9,11,16,17],
              11:[3,4,10,12,17,18],
              12:[4,5,11,13,18,19],
              13:[5,6,12,14,19,20],
              14:[6,7,8,13,20,21],
              15:[8,9,16,21,22,23],
              16:[9,10,15,17,23,24],
              17:[10,11,16,18,24,25],
              18:[11,12,17,19,25,26],
              19:[12,13,18,20,26,27],
              20:[13,14,19,21,27,28],
              21:[8,14,15,20,22,28],
              22:[15,21,23,28,29,35],
              23:[15,16,22,24,29,30],
              24:[16,17,23,25,30,31],
              25:[17,18,24,26,31,32],
              26:[18,19,25,27,32,33],
              27:[19,20,26,28,33,34],
              28:[20,21,22,27,34,35],
              29:[22,23,30,35,36,37],
              30:[23,24,29,31,37,38],
              31:[24,25,30,32,38,39],
              32:[25,25,31,33,39,40],
              33:[26,27,32,34,40,41],
              34:[27,28,33,35,41,42],
              35:[22,28,29,34,36,42],
              36:[29,35,37,42,43,49],
              37:[29,30,36,38,43,44],
              38:[30,31,37,39,44,45],
              39:[31,32,38,40,45,46],
              40:[32,33,39,41,46,47],
              41:[33,34,40,42,47,48],
              42:[34,35,41,36,48,49],
              43:[4,5,36,37,44,49],
              44:[5,6,37,38,43,45],
              45:[6,7,38,39,44,46],
              46:[1,7,39,40,45,47],
              47:[1,2,40,41,46,48],
              48:[2,3,41,42,47,49],
              49:[3,4,36,42,43,48]}

loop_time = 25
Pb = np.zeros(loop_time)
Pd = np.zeros(loop_time)

for i in range(loop_time):
    Pb[i], Pd[i] = markov_chain_cellular_network(num_cell, total_arrivals, mu, lam, handover_probability, delta, k, neighbours)

prob_blocking_confidence_interval = stats.norm.interval(alpha = 0.95,
                                                        loc = np.mean(Pb),
                                                        scale = stats.sem(Pb))
print("Blocking Probability (Markov Chain):", np.mean(Pb))
print("Blocking Probability Confidence Interval (Markov Chain): ", prob_blocking_confidence_interval)

Blocking Probability (Markov Chain): 0.0008052
Blocking Probability Confidence Interval (Markov Chain):  (0.0007695753518549943, 0.0008408246481450055)
